In [1]:
import json
import numpy as np
import os
from config import cfg, get_emb
import progressbar

In [2]:
emb_size = 300
w2v_name = 'Glove' # 'BERT' # 

In [3]:
if w2v_name == 'BERT':
    emb_file = '{}/word-embedding/bert.PCA.300d.txt'.format(cfg.DATA_DIR)
elif w2v_name == 'Glove':
    emb_file = '{}/word-embedding/glove.6B.300d.txt'.format(cfg.DATA_DIR)
emb_dict, mean, std = get_emb(emb_file, emb_size)

In [4]:
split = 'TVA' # train on Tr; test on Val
# split = 'TVEA' # train on Tr+Val; test on Te

# suffix = '' # ocr_bert_DR
# suffix = '.0init' # ocr_bert_DR.0init
# suffix = '.clean.0init' # ocr_bert_DR.clean.0init
# suffix = '.assemble.0init'
#=======================================
suffix = ''
zeroInit = True # True for 0 init; False for random init
if zeroInit:
    suffix += '.0init'

In [5]:
jsdict = json.load(open('{}/data.json'.format(cfg.DATA_DIR),'r'))
print(jsdict.keys())
itoa = jsdict['itoa']
removeIdx = -1
for i,a in enumerate(itoa):
    if a == '':
        print('index:({}), content:({})'.format(i,a))
        itoa[i] = '<blank>'
        removeIdx = i

dict_keys(['itoa', 'itow'])
index:(20), content:()


In [6]:
if zeroInit:
    ans_emb = np.zeros((len(itoa), emb_size), dtype='float32')
else:
    ans_emb = np.random.randn(len(itoa), emb_size).astype('float32')
    ans_emb = (ans_emb + mean) * std
print(ans_emb.shape)

(8205, 300)


In [7]:
bar = progressbar.ProgressBar(max_value=len(itoa)).start()
fill_cnt = 0
for i,ans in enumerate(itoa):
    bar.update(i+1)
    words = ans.split()
    act = 0
    emb = np.zeros((emb_size,), dtype='float32')
    for w in words:
        if w in emb_dict:
            act += 1
            emb += emb_dict[w]
    if act > 0:
        emb /= act
        ans_emb[i] = emb
        fill_cnt += 1
bar.finish()
print('fill embedding: {}/{}'.format(fill_cnt, len(itoa)))

100% (8205 of 8205) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


fill embedding: 7417/8205


In [8]:
if w2v_name == 'BERT':
    save_folder = '{}/bertOld.{}{}/{}'.format(cfg.OCR_DIR, emb_size, suffix, split)
elif w2v_name == 'Glove':
    save_folder = '{}/glove.{}{}/{}'.format(cfg.OCR_DIR, emb_size, suffix, split)
save_path = '{}/ans_emb.npy'.format(save_folder)
print('[Save] save path: {}'.format(save_path))
if not os.path.isdir(save_folder):
    os.makedirs(save_folder)
np.save(save_path, ans_emb)

[Save] save path: /home/lyt/code/concept_vqa/dataTVQA/image-feature/ocr/glove.300.0init/TVA/ans_emb.npy
